In [1]:
import io
import os
import json

# Imports the Google Cloud client library
from google.cloud import vision_v1p1beta1
from google.cloud.vision_v1p1beta1 import types

In [2]:
%env GOOGLE_APPLICATION_CREDENTIALS=/Users/chappers/Documents/DataScience/0_Method_DS/MethodDS-KaggleKat-8b57553f8dbf.json

env: GOOGLE_APPLICATION_CREDENTIALS=/Users/chappers/Documents/DataScience/0_Method_DS/MethodDS-KaggleKat-8b57553f8dbf.json


In [3]:
# Instantiates a client
client = vision_v1p1beta1.ImageAnnotatorClient()

# The name of the image file to annotate
file_name = os.path.join(
    os.path.dirname('__file__'),
    'IMG_2934.jpg')

In [4]:
# Loads the image into memory
with io.open(file_name, 'rb') as image_file:
    content = image_file.read()

image = types.Image(content=content)

# Performs label detection on the image file
response = client.label_detection(image=image)
labels = response.label_annotations

print('Labels:')
for label in labels:
    # print(label.description)
    print(label)

Labels:
mid: "/m/0n5v01m"
description: "bag"
score: 0.8551287651062012
topicality: 0.8551287651062012

mid: "/m/01jwgf"
description: "product"
score: 0.7906313538551331
topicality: 0.7906313538551331

mid: "/m/080hkjn"
description: "handbag"
score: 0.7440301775932312
topicality: 0.7440301775932312

mid: "/m/04t7l"
description: "metal"
score: 0.6029984354972839
topicality: 0.6029984354972839

mid: "/m/03hzcs"
description: "beige"
score: 0.587562084197998
topicality: 0.587562084197998

mid: "/m/0j62f"
description: "rectangle"
score: 0.5806199908256531
topicality: 0.5806199908256531

mid: "/m/0hgrzkp"
description: "shoulder bag"
score: 0.5463389158248901
topicality: 0.5463389158248901



In [5]:
# create str_search to be used in the EBAY API call for KEYWORDS
# str_search = str(labels[0].description + " , " +
#                 labels[2].description + " , " +
#                 # + labels[3].description + " , " +
#                 labels[4].description + " , " + 
                 # labels[5].description + " , " +
#                 labels[6].description)
str_search = str(labels[0].description)

In [6]:
web_scores=[]
best_g=[]
# def annotate(path):
#    """Returns web annotations given the path to an image."""
#    client = vision.ImageAnnotatorClient()

#    if path.startswith('http') or path.startswith('gs:'):
#        image = types.Image()
#        image.source.image_uri = path

#    else:
#        with io.open(path, 'rb') as image_file:
#            content = image_file.read()

#        image = vision.types.Image(content=content)

#    web_detection = client.web_detection(image=image).web_detection

#    return web_detection

def detect_web(path):
    """Detects web annotations given an image."""
    client = vision_v1p1beta1.ImageAnnotatorClient()

    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision_v1p1beta1.types.Image(content=content)

    # how doe we get this 
    response = client.web_detection(image=image)
    annotations = response.web_detection

    for label in annotations.best_guess_labels:
        print('\nBest guess label: {}'.format(label.label))
        best_g.append(format(label.label))

# def report(annotations):
    """Prints detected features in the provided web annotations."""
    if annotations.pages_with_matching_images:
        print('\n{} Pages with matching images retrieved'.format(
            len(annotations.pages_with_matching_images)))

        for page in annotations.pages_with_matching_images:
            print('Url   : {}'.format(page.url))
            
#    if annotations.best_guess_labels:
#        for label in annotations.best_guess_labels:
#            print('\nBest guess label: {}'.format(label.label))

    if annotations.full_matching_images:
        print ('\n{} Full Matches found: '.format(
               len(annotations.full_matching_images)))

        for image in annotations.full_matching_images:
            print('Url  : {}'.format(image.url))

    if annotations.partial_matching_images:
        print ('\n{} Partial Matches found: '.format(
               len(annotations.partial_matching_images)))

        for image in annotations.partial_matching_images:
            print('Url  : {}'.format(image.url))

    if annotations.web_entities:
        print ('\n{} Web entities found: '.format(
            len(annotations.web_entities)))

        for entity in annotations.web_entities:
            print('Score      : {}'.format(entity.score))
            print('Description: {}'.format(entity.description))
            if (entity.score>0.90):
                web_scores.append(entity.description)

In [7]:
# report(annotate('IMG_2934.jpg'))
detect_web('IMG_2934.jpg')


Best guess label: michael kors travel bag

10 Web entities found: 
Score      : 1.2195520401000977
Description: Handbag
Score      : 0.9519000053405762
Description: Michael Kors
Score      : 0.7020999789237976
Description: Tote bag
Score      : 0.7020000219345093
Description: Bag
Score      : 0.6345999836921692
Description: 
Score      : 0.612661600112915
Description: Product design
Score      : 0.5934000015258789
Description: Messenger Bags
Score      : 0.4374000132083893
Description: Design
Score      : 0.43650001287460327
Description: Travel
Score      : 0.41990000009536743
Description: Shoulder


In [8]:
for rows in range(0,len(web_scores)):
    str_search=(str_search+" , "+ web_scores[rows])

In [9]:
keyword_string = best_g[0]
keyword_string

'michael kors travel bag'

In [28]:
# call EBAY API and get offers, based on the STR_SEARCH value.
# note, 
from ebaysdk.finding import Connection as Finding
from ebaysdk.exception import ConnectionError

try:
    api = Finding(siteid='EBAY-US', appid="MethodDa-MethodDa-PRD-babd43d69-a4097943", config_file=None)
    response = api.execute('findItemsAdvanced', {'keywords': keyword_string})
    # print(response.dict())
except ConnectionError as e:
    print(e)
    print(e.response.dict())

In [29]:
with open('image_G_output.json', 'w') as outfile:
    json.dump(response.dict(), outfile)

In [44]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content,'lxml')

In [68]:
totalentries = int(soup.find('totalentries').text)
items = soup.find_all('item')
item_id = soup.find_all('itemid')
item_title = soup.find_all('title')
item_url = soup.find_all('viewitemurl')
item_price = soup.find_all('currentprice')
for rows in range(0,len(items)):
                  print('ID = ' + str(item_id[rows].text) + " : " + str(item_title[rows].text) + '\n' + 
                       str(item_url[rows].text) + '\n' +
                       str(item_price[rows].text + '\n'))

ID = 352329079584 : NWT Michael Kors Blossom Saffiano Leather Jet Set Travel Chain Shoulder Tote Bag
http://www.ebay.com/itm/NWT-Michael-Kors-Blossom-Saffiano-Leather-Jet-Set-Travel-Chain-Shoulder-Tote-Bag-/352329079584
166.66

ID = 292441042622 : MICHAEL KORS Fragrances gold metallic tote bag shopper purse travel NEW
http://www.ebay.com/itm/MICHAEL-KORS-Fragrances-gold-metallic-tote-bag-shopper-purse-travel-NEW-/292441042622
27.95

ID = 173367109356 : MICHAEL KORS JET SET TRAVEL CARRYALL TOTE FLORAL PVC BAG NAVY 
http://www.ebay.com/itm/MICHAEL-KORS-JET-SET-TRAVEL-CARRYALL-TOTE-FLORAL-PVC-BAG-NAVY-/173367109356
105.99

ID = 123188918736 : MICHAEL KORS gold metallic duffle bag travel gym weekender overnight tote
http://www.ebay.com/itm/MICHAEL-KORS-gold-metallic-duffle-bag-travel-gym-weekender-overnight-tote-/123188918736
45.0

ID = 302753879781 : Michael Kors Jet Set Travel Messenger Large Crossbody Bag Signature Vanilla
http://www.ebay.com/itm/Michael-Kors-Jet-Set-Travel-Messenger-La